In [ ]:
import os

In [3]:
%pwd

'/Users/suyash/Desktop/End to End/research'

In [4]:
os.chdir('../')

In [5]:
import pandas as pd

In [8]:
from dataclasses import dataclass
from pathlib import Path    


@dataclass
class DataSpilterConfig:
    root_dir:Path
    train_data:Path
    test_data:Path
    split_ratio:float
    random_state:int

In [9]:
from src.IPL_Predictor.constants import *
from src.IPL_Predictor.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_spilter_config(self) -> DataSpilterConfig:
        config = self.config.data_spilter
        split_ratio = self.config.split_ratio
        random_state = self.config.random_state

        create_directories([config.root_dir])

        data_spilter_config = DataSpilterConfig(
            root_dir=config.root_dir,
            train_data=config.train_data,
            test_data=config.test_data,
            split_ratio=config.split_ratio,
            random_state=config.random_state,
        )
        return data_spilter_config
        

In [16]:
from src.IPL_Predictor import logger

In [17]:
class DataSpilter:
    def __init__(self, config: DataSpilterConfig):
        self.config = config

    def spilt_data(self):
        df = pd.read_csv(self.config.unzip_data_dir)


        train_df = df.sample(
            frac=1 - self.config.split_ratio,
            random_state=self.config.random_state
        )
        test_df = df.drop(train_df.index)

        train_df.to_csv(self.config.train_data, index=False)
        test_df.to_csv(self.config.test_data, index=False)

        logger.info(f"Train and Test data saved in {self.config.root_dir}")

        return self.config.train_data, self.config.test_data

In [18]:
try:
    config = ConfigurationManager()
    data_spiltting_config = config.get_data_spilter_config()
    data_validation = DataSpilter(config=data_spiltting_config)
    data_validation.spilt_data()
except Exception as e:
    logger.error(f"Error occurred during data validation: {e}")

[2025-09-21 18:13:35,422: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-21 18:13:35,424: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-21 18:13:35,426: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-21 18:13:35,427: INFO: common: created directory at : artifacts]
[2025-09-21 18:13:35,427: ERROR: 353886353: Error occurred during data validation: "'ConfigBox' object has no attribute 'data_spilter'"]
